In [3]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

In [4]:
import pandas as pd
from src.models import fit_ols, extract_coefficients
from src.config import DATA_PROCESSED

In [5]:
df = pd.read_csv(DATA_PROCESSED / "base_analitica.csv", parse_dates=["date"])

y = "ln_demanda"

X = [
    "ln_tarifa",
    "ln_pib",
    "ln_importacoes",
    "d_2001",
    "d_2006",
    "d_2008",
    "d_2020"
]

results = fit_ols(df, y, X)

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             ln_demanda   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     408.1
Date:                Wed, 14 Jan 2026   Prob (F-statistic):          1.20e-160
Time:                        16:58:57   Log-Likelihood:                 534.53
No. Observations:                 346   AIC:                            -1053.
Df Residuals:                     338   BIC:                            -1022.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             11.9793      0.257     46.